In [1]:
import re
import time
import json
import os
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# ================= Configuration =================
TARGET_YEARS = list(range(2025, 2024 , -1))  # Target Years, including 2025
MIN_VOTES = 10000
MIN_BOX_OFFICE = 50000000  # Minimum Box Office in USD with at least 50 million

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}
# =================================================


In [3]:
def get_high_res_poster(url):
    if not url: return ""
    return re.sub(r'V1_.*?.jpg', 'V1_.jpg', url)

def format_date_output(date_obj):
    return date_obj.strftime("%b %-d, %Y")

def get_movie_detail_date(driver, imdb_id):
    if not imdb_id: return None
    url = f"https://www.imdb.com/title/{imdb_id}/"
    try:
        driver.get(url)
        time.sleep(1.2) 
        
        soup = BeautifulSoup(driver.page_source, "lxml")
        
        json_ld_script = soup.find('script', type='application/ld+json')
        if json_ld_script:
            try:
                data = json.loads(json_ld_script.string)
                if 'datePublished' in data:
                    dt = datetime.strptime(data['datePublished'], "%Y-%m-%d")
                    return format_date_output(dt)
            except: pass

        release_date_item = soup.find("a", {"href": re.compile(r"releaseinfo")})
        if release_date_item:
            clean_text = re.split(r'\s*\(', release_date_item.get_text(strip=True))[0].strip()
            try:
                dt = datetime.strptime(clean_text, "%B %d, %Y")
                return format_date_output(dt)
            except: pass
        return None
    except Exception: return None

def sort_movies_by_date(movies_list, reverse=False):
    def parse_date(movie):
        try:
            return datetime.strptime(movie['date'], "%b %d, %Y")
        except ValueError:
            return datetime(1, 1, 1)
    movies_list.sort(key=parse_date, reverse=reverse)
    return movies_list

def load_all_results(driver):
    print("   └── Loading full list (clicking '50 more')...")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        try:
            more_button = WebDriverWait(driver, 4).until(
                EC.presence_of_element_located((By.CLASS_NAME, "ipc-see-more__button"))
            )
            driver.execute_script("arguments[0].click();", more_button)
            print("      [+] Clicked '50 more' button...")
            time.sleep(3)
        except Exception:
            print("      [✓] List fully loaded (No more buttons).")
            break

def scrape_year(driver, year, min_votes, min_box_office):
    print(f"\n🎬 Scanning Year: {year}...")
    
    url = (
        f"https://www.imdb.com/search/title/?"
        f"title_type=feature"
        f"&release_date={year}-01-01,{year}-12-31"
        f"&num_votes={min_votes},"
        f"&gross={min_box_office},"
        f"&sort=boxoffice_gross_us,desc"
    )
    
    driver.get(url)
    time.sleep(3) 

    load_all_results(driver)
    
    print("   ⏳ Extracting HTML source code...")
    html_source = driver.page_source
    
    print("   ⏳ Parsing HTML with lxml...")
    soup = BeautifulSoup(html_source, "lxml") 
    
    movie_items = soup.select("li.ipc-metadata-list-summary-item")
    total_found = len(movie_items)
    
    print(f"   📊 Statistics for {year}:")
    print(f"      - Total Qualified Movies Found: {total_found}")
    
    movies_list = []
    
    for index, item in enumerate(movie_items):
        title_tag = item.select_one("h3.ipc-title__text")
        if not title_tag: continue
        clean_title = title_tag.get_text(strip=True).split('. ', 1)[-1]
        
        img_tag = item.select_one("img.ipc-image")
        poster_url = get_high_res_poster(img_tag.get("src")) if img_tag else ""
        
        link_tag = item.select_one("a.ipc-title-link-wrapper")
        imdb_id = re.search(r'/title/(tt\d+)/', link_tag.get("href")).group(1) if link_tag else ""
        
        print(f"      [{index+1}/{total_found}] Processing: {clean_title} ...")
        
        specific_date = get_movie_detail_date(driver, imdb_id)
        final_date = specific_date if specific_date else f"Jan 1, {year}"
        
        movies_list.append({
            "title": clean_title,
            "date": final_date,
            "poster": poster_url
        })
        
    sorted_movies = sort_movies_by_date(movies_list, reverse=False)
    
    saved_count = len(sorted_movies)
    print(f"   ✅ Year {year} processing finished. Ready to save.")

    return {
        "year": year,
        "total_count": total_found,
        "saved_count": saved_count,
        "favorite": sorted_movies[0]['title'] if sorted_movies else "",
        "movies": sorted_movies
    }

def run_imdb_scraper(target_years, min_votes=10000, min_box_office=50000000):
    chrome_options = Options()
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--log-level=3")
    
    print("🚀 Starting Browser...")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        for year in target_years:
            year_data = scrape_year(driver, year, min_votes, min_box_office)
            
            if year_data:
                output_file = f"{year}.json"
                print(f"   💾 Saving {output_file}...")
                with open(output_file, "w", encoding="utf-8") as f:
                    json.dump(year_data, f, indent=2, ensure_ascii=False)
                print(f"   🎉 Saved {year} data successfully.")
                
        print(f"\n🏁 All years processed.")
        
    finally:
        driver.quit()


In [4]:

def process_movie_files():
    for filename in os.listdir('.'):
        if filename.endswith('.json') and filename[:-5].isdigit():
            target_year = int(filename[:-5])
            
            try:
                with open(filename, 'r', encoding='utf-8') as f:
                    data = json.load(f)
            except Exception:
                continue

            if not isinstance(data, dict) or 'movies' not in data:
                continue

            valid_movies = []
            for movie in data['movies']:
                date_str = movie.get('date', '')
                try:
                    dt_obj = datetime.strptime(date_str, "%b %d, %Y")
                    if dt_obj.year == target_year:
                        movie['_sort_key'] = dt_obj
                        valid_movies.append(movie)
                except ValueError:
                    continue

            valid_movies.sort(key=lambda x: x['_sort_key'])

            for movie in valid_movies:
                del movie['_sort_key']

            data['movies'] = valid_movies
            data['total_count'] = len(valid_movies)
            data['saved_count'] = len(valid_movies)
            
            if valid_movies:
                data['favorite'] = valid_movies[0].get('title', '')
            
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)


In [5]:
print("=================================================")
print(f"🚀 任务启动 | 目标年份: {TARGET_YEARS}")
print(f"🎯 筛选门槛 | 票房 > ${MIN_BOX_OFFICE:,} | 票数 > {MIN_VOTES:,}")
print("=================================================")

run_imdb_scraper(TARGET_YEARS, MIN_VOTES, MIN_BOX_OFFICE)

print("\n🧹 正在执行数据清洗与校对...")
process_movie_files()

print("\n✨ 全部流程结束！请查看生成的 .json 文件。")


🚀 任务启动 | 目标年份: [2025]
🎯 筛选门槛 | 票房 > $50,000,000 | 票数 > 10,000
🚀 Starting Browser...

🎬 Scanning Year: 2025...
   └── Loading full list (clicking '50 more')...
      [+] Clicked '50 more' button...
      [+] Clicked '50 more' button...
      [+] Clicked '50 more' button...
      [+] Clicked '50 more' button...
      [+] Clicked '50 more' button...
      [✓] List fully loaded (No more buttons).
   ⏳ Extracting HTML source code...
   ⏳ Parsing HTML with lxml...
   📊 Statistics for 2025:
      - Total Qualified Movies Found: 252
      [1/252] Processing: A Minecraft Movie ...
      [2/252] Processing: Lilo & Stitch ...
      [3/252] Processing: Zootopia 2 ...
      [4/252] Processing: Superman ...
      [5/252] Processing: Avatar: Fire and Ash ...
      [6/252] Processing: Wicked: For Good ...
      [7/252] Processing: Jurassic World: Rebirth ...
      [8/252] Processing: Sinners ...
      [9/252] Processing: The Fantastic Four: First Steps ...
      [10/252] Processing: How to Train Your 